In [101]:
import datetime as dt
from lxml import html
from bs4 import BeautifulSoup
import urllib
import sys
import requests
import operator
from datetime import timedelta
import csv
import pandas as pd
import numpy as np
import math

In [102]:
year1617 = pd.read_csv('getting_SOS_games/season1617.csv', parse_dates=['Date'])
year1718 = pd.read_csv('getting_SOS_games/season1718.csv', parse_dates=['Date'])
year1819 = pd.read_csv('getting_SOS_games/season1819.csv', parse_dates=['Date'])
year1920 = pd.read_csv('getting_SOS_games/season1920.csv', parse_dates=['Date'])

In [103]:
frames = [year1617,year1718,year1819,year1920]

In [104]:
full = pd.concat(frames).reset_index(drop=True)

In [105]:
full['KenPomVal_Away'].min(),full['KenPomVal_Away'].quantile(.25),full['KenPomVal_Away'].quantile(.5),full['KenPomVal_Away'].quantile(.75),full['KenPomVal_Away'].max()

(-33.99, -8.87, -1.25, 7.03, 34.22)

In [106]:
top_games = full[(full['KenPomVal_Away']<-8.87) & (full['KenPomVal_Home']<-8.87)]
top_games = top_games.dropna().reset_index(drop=True)

In [107]:
y = 0
for x in top_games.columns:
    print(x,y)
    y+=1

Unnamed: 0 0
Date 1
Win_Score 2
Lose_Score 3
Home 4
Away 5
WinnerHome 6
KenPomRank_Away 7
KenPomVal_Away 8
BPIrank_Away 9
BPIval_Away 10
SRS_Away 11
SOS_Away 12
o_eff_Away 13
d_eff_Away 14
ppg_Away 15
papg_Away 16
Pace_Away 17
Three_Par_Away 18
True_Shoot_Away 19
Tov_Away 20
Opp_Three_Par_Away 21
Opp_True_Shoot_Away 22
Opp_Tov_Away 23
KenPomRank_Home 24
KenPomVal_Home 25
BPIrank_Home 26
BPIval_Home 27
SRS_Home 28
SOS_Home 29
o_eff_Home 30
d_eff_Home 31
ppg_Home 32
papg_Home 33
Pace_Home 34
Three_Par_Home 35
True_Shoot_Home 36
Tov_Home 37
Opp_Three_Par_Home 38
Opp_True_Shoot_Home 39
Opp_Tov_Home 40


In [108]:
top_games

,Unnamed: 0,Date,Win_Score,Lose_Score,Home,Away,WinnerHome,KenPomRank_Away,KenPomVal_Away,BPIrank_Away,...,d_eff_Home,ppg_Home,papg_Home,Pace_Home,Three_Par_Home,True_Shoot_Home,Tov_Home,Opp_Three_Par_Home,Opp_True_Shoot_Home,Opp_Tov_Home
0,35,2016-11-11,77,68,Pepperdine,Cal Poly,1,306.0,-12.93,311.0,...,112.15,69.0,80.3,69.8,0.266,0.517,15.8,0.372,0.585,13.7
1,36,2016-11-11,71,55,Portland,UC-Riverside,1,311.0,-13.63,299.0,...,104.78,67.1,72.2,67.5,0.320,0.526,16.8,0.413,0.531,14.5
2,47,2016-11-11,84,73,South Florida,Florida A&M,1,345.0,-22.61,345.0,...,105.01,64.8,74.5,69.2,0.299,0.504,20.2,0.411,0.544,16.2
3,153,2016-11-13,80,58,Longwood,Maine,0,337.0,-19.62,333.0,...,116.84,67.3,80.3,71.0,0.342,0.533,19.4,0.343,0.570,15.2
4,197,2016-11-14,64,63,Hawaii,Florida Atlantic,1,271.0,-9.33,290.0,...,103.02,68.1,68.3,68.3,0.399,0.527,17.6,0.330,0.535,18.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590,5258,2020-03-10,70,63,South Carolina State,Howard,0,350.0,-26.15,348.0,...,93.25,18.0,70.2,69.1,0.299,0.530,17.9,0.356,0.552,15.4
1591,5261,2020-03-10,56,54,Niagara,Marist,1,317.0,-13.59,320.0,...,96.61,20.0,65.7,66.5,0.366,0.528,14.5,0.410,0.553,17.5
1592,5263,2020-03-10,68,64,Delaware State,Maryland-Eastern Shore,1,349.0,-26.14,350.0,...,90.16,26.0,73.4,76.3,0.395,0.507,17.4,0.406,0.573,16.9
1593,5273,2020-03-11,92,75,North Carolina Central,Delaware State,1,346.0,-23.11,347.0,...,91.91,13.0,69.7,69.6,0.349,0.537,19.0,0.388,0.507,20.9


In [120]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [121]:
full = top_games[['Home','Away','WinnerHome','KenPomVal_Away','BPIval_Away','SRS_Away','KenPomVal_Home','BPIval_Home','SRS_Home']]
X = full.iloc[:,[3,4,5,6,7,8]]
y = full.iloc[:,2]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=14)
LR = LogisticRegression().fit(X_train, y_train)

In [122]:
LR.score(X_test, y_test)

0.6701461377870563

In [123]:
import pandas as pd
feature_imp = pd.Series(LR.coef_[0],index=X.columns).sort_values(ascending=False)
feature_imp

KenPomVal_Home    0.140062
BPIval_Away       0.023937
BPIval_Home       0.020109
SRS_Away          0.005236
SRS_Home         -0.006515
KenPomVal_Away   -0.163647
dtype: float64

In [124]:
X = top_games.iloc[:,[12,18,19,20,21,22,23,29,35,36,37,38,39,40]]
y = top_games.iloc[:,6]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=14)
LR = LogisticRegression(max_iter=300).fit(X_train, y_train)

In [125]:
LR.score(X_test, y_test)

0.6283924843423799

In [126]:
import pandas as pd
feature_imp = pd.Series(LR.coef_[0],index=X.columns).sort_values(ascending=False)
feature_imp

True_Shoot_Home        2.028620
Opp_True_Shoot_Away    1.633436
Opp_Tov_Home           0.126811
Tov_Away               0.069113
SOS_Home               0.058916
Three_Par_Away         0.020994
SOS_Away              -0.043607
Opp_Tov_Away          -0.109116
Tov_Home              -0.119562
Opp_Three_Par_Away    -0.279772
Three_Par_Home        -0.385334
Opp_Three_Par_Home    -0.997296
True_Shoot_Away       -1.650313
Opp_True_Shoot_Home   -1.917521
dtype: float64